## Create an invoice generator for a business with list of items and prices

In [ ]:
from abc import ABC, abstractmethod
from datetime import datetime, timedelta
import re

class Business(ABC):
    def __init__(self, companyName, companyAddress, phoneNumber, email, gstin, CIN):
        self._companyName = companyName
        self.companyAddress = companyAddress
        self._phoneNumber = phoneNumber
        self._email = email
        self._gstin = gstin
        self._CIN = CIN

    @property
    def companyName(self):
        return self._companyName.title()

    @companyName.setter
    def companyName(self, value):
        if len(value) >= 2:
            self._companyName = value
        else:
            raise ValueError("Company Name must be at least 2 characters.")

    @property
    def email(self):
        return self._email.lower()

    @email.setter
    def email(self, value):
        regex = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b"
        if re.fullmatch(regex, value):
            self._email = value
        else:
            raise ValueError("Invalid email format.")

    @property
    def phoneNumber(self):
        return self._phoneNumber

    @phoneNumber.setter
    def phoneNumber(self, value):
        if len(value) == 10 and value.isdigit():
            self._phoneNumber = value
        else:
            raise ValueError("Phone must be 10 digits.")

    @property
    def gstin(self):
        return self._gstin.upper()

    @gstin.setter
    def gstin(self, value):
        if len(value) == 15:
            self._gstin = value
        else:
            raise ValueError("GSTIN must be 15 digits.")

    def contact_info(self):
        return f"Email: {self.email} | Phone: {self.phoneNumber}"


class Freelancer(Business):
    def __init__(
        self,
        companyName,
        companyAddress,
        phoneNumber,
        email,
        gstin,
        CIN,
        founderName,
        website,
        PAN,
        services,
    ):
        super().__init__(companyName, companyAddress, phoneNumber, email, gstin, CIN)
        self.founderName = founderName
        self.website = website
        self._PAN = PAN
        self.services = services

    @property
    def website(self):
        return self._website.lower()

    @website.setter
    def website(self, value):
        regex = r"^(https?:\/\/)?([\w\-]+\.)+[\w\-]+(\/[\w\-]*)*$"
        if re.fullmatch(regex, value):
            self._website = value
        else:
            raise ValueError("Invalid website format.")

    @property
    def PAN(self):
        return self._PAN.upper()

    @PAN.setter
    def PAN(self, value):
        regex = r"[A-Z]{5}[0-9]{4}[A-Z]"
        if re.fullmatch(regex, value):
            self._PAN = value
        else:
            raise ValueError("Invalid PAN format.")

    def add_service(self, name, rate):
        self.services[name] = rate

    def view_services(self):
        print("====== Service Catalog ======")
        for name, rate in self.services.items():
            print(f"{name:<25} ₹{rate:.2f}")
        print("=============================")

    def update_service(self, name, new_rate):
        if name in self.services:
            self.services[name] = new_rate
        else:
            print(f"Service '{name}' not found.")

    def delete_service(self, name):
        if name in self.services:
            del self.services[name]
        else:
            print(f"Service '{name}' not found.")


class Client(Business):
    def __init__(self, companyName, companyAddress, phoneNumber, email, gstin, CIN, ID):
        super().__init__(companyName, companyAddress, phoneNumber, email, gstin, CIN)
        self.ID = ID

    @property
    def CIN(self):
        return self._CIN.upper()

    @CIN.setter
    def CIN(self, value):
        if len(value) == 21:
            self._CIN = value
        else:
            raise ValueError("CIN must be 21 characters.")


class Invoice:
    cgstRate = 9
    sgstRate = 9

    def __init__(self, freelancer, client):
        self.freelancer = freelancer
        self.client = client
        self.items = {}  # {service_name: (rate, qty)}

    def add_service_to_invoice(self, service_name, quantity):
        if service_name in self.freelancer.services:
            rate = self.freelancer.services[service_name]
            self.items[service_name] = (rate, quantity)
        else:
            raise ValueError(f"Service '{service_name}' not found in catalog.")

    def calculate_total(self):
        return sum(rate * qty for rate, qty in self.items.values())

    @staticmethod
    def tax(total):
        cgst = round((total * Invoice.cgstRate / 100), 2)
        sgst = round((total * Invoice.sgstRate / 100), 2)
        return {"cgst": cgst, "sgst": sgst, "total_tax": round(cgst + sgst, 2)}

    @staticmethod
    def invoiceDetails(client_id):
        now = datetime.now()
        invoice_no = f"INV-{now.strftime('%Y%m%d')}-{client_id}"
        invoice_date = now.strftime("%d-%m-%Y")
        due_date = (now + timedelta(days=7)).strftime("%d-%m-%Y")
        return {
            "invoice_no": invoice_no,
            "invoice_date": invoice_date,
            "due_date": due_date,
        }

    @classmethod
    def updateCGST(cls, value):
        if value >= 0:
            cls.cgstRate = value
        else:
            raise ValueError("CGST can't be negative.")

    @classmethod
    def updateSGST(cls, value):
        if value >= 0:
            cls.sgstRate = value
        else:
            raise ValueError("SGST can't be negative.")